## 1. Import Libraries

In [120]:
import numpy as np
import pandas as pd


import sklearn
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.preprocessing import (
    OneHotEncoder, 
    MinMaxScaler, 
    StandardScaler,
    PowerTransformer,
    FunctionTransformer
)

from feature_engine.outliers import Winsorizer
from feature_engine.encoding import (
    RareLabelEncoder,
    MeanEncoder, 
    CountFrequencyEncoder
)
from feature_engine.datetime import DatetimeFeatures

## 2. Display Settings

In [2]:
pd.set_option('display.max_columns',None)

>- Display settings force pandas to show all the columns each time if we import the data and vast columns we have then it can show less columns.

In [3]:
sklearn.set_config(transform_output='pandas')

>- So each times if we given an dataframe to sickit learn output returns in numpy array so this snippet helps to see that putput in dataframe itself.

In [4]:
import warnings
warnings.filterwarnings("ignore")

## 3. Read The Data

In [5]:
path = r'C:\Users\admin\OneDrive\Desktop\Flights_SageMaker_Project\Data\train.csv'

train = pd.read_csv(path)
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-06,Delhi,Cochin,14:20:00,22:30,490,1.0,No Info,6938
1,Multiple Carriers,2019-06-15,Delhi,Cochin,11:30:00,19:15,465,1.0,No Info,16108
2,Jet Airways,2019-05-12,Kolkata,Banglore,17:00:00,22:05,1745,1.0,No Info,13067
3,Air India,2019-05-09,Delhi,Cochin,14:25:00,19:15,1730,2.0,No Info,13591
4,Air India,2019-03-06,Mumbai,Hyderabad,12:45:00,19:25,1840,2.0,No Info,13253
...,...,...,...,...,...,...,...,...,...,...
635,Multiple Carriers,2019-05-18,Delhi,Cochin,10:20:00,19:00,520,1.0,No Info,9794
636,Jet Airways,2019-05-21,Delhi,Cochin,08:00:00,04:25,1225,1.0,In-flight meal not included,12898
637,Jet Airways,2019-05-24,Kolkata,Banglore,09:35:00,09:45,1450,1.0,No Info,13067
638,Indigo,2019-06-12,Banglore,Delhi,07:10:00,10:05,175,0.0,No Info,4823


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   airline          640 non-null    object 
 1   date_of_journey  640 non-null    object 
 2   source           640 non-null    object 
 3   destination      640 non-null    object 
 4   dep_time         640 non-null    object 
 5   arrival_time     640 non-null    object 
 6   duration         640 non-null    int64  
 7   total_stops      640 non-null    float64
 8   additional_info  640 non-null    object 
 9   price            640 non-null    int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 50.1+ KB


In [7]:
X_train = train.drop(columns='price')
y_train = train['price'].copy()

## 4. Transformation Operations

### 4.1 Airline

In [8]:
X_train.airline

0                 Indigo
1      Multiple Carriers
2            Jet Airways
3              Air India
4              Air India
             ...        
635    Multiple Carriers
636          Jet Airways
637          Jet Airways
638               Indigo
639          Jet Airways
Name: airline, Length: 640, dtype: object

In [9]:
air_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("grouper", RareLabelEncoder(tol=0.1, replace_with='other', n_categories=2)),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

air_transformer.fit_transform(train.loc[:,['airline']])#.columns.str.replace(" ","_")

,airline_Air India,airline_Indigo,airline_Jet Airways,airline_Multiple Carriers,airline_other
0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
635,0.0,0.0,0.0,1.0,0.0
636,0.0,0.0,1.0,0.0,0.0
637,0.0,0.0,1.0,0.0,0.0
638,0.0,1.0,0.0,0.0,0.0


>- tol :Tolerance used in RareLabelEncoder that helps to group the categories which are less than 10%

### 4.2 date_of_journey

In [10]:
X_train.date_of_journey

0      2019-06-06
1      2019-06-15
2      2019-05-12
3      2019-05-09
4      2019-03-06
          ...    
635    2019-05-18
636    2019-05-21
637    2019-05-24
638    2019-06-12
639    2019-06-15
Name: date_of_journey, Length: 640, dtype: object

In [21]:
feature_to_extract = ['month','week', 'day_of_week','day_of_year']

doj_transformer = Pipeline(steps=[
    ('dt', DatetimeFeatures(features_to_extract=feature_to_extract, yearfirst=True, format='mixed')),
    ("scaler", MinMaxScaler())
])

doj_transformer.fit_transform(X_train[['date_of_journey']])#.date_of_journey_week.value_counts()

,date_of_journey_month,date_of_journey_week,date_of_journey_day_of_week,date_of_journey_day_of_year
0,1.000000,0.823529,0.500000,0.822034
1,1.000000,0.882353,0.833333,0.898305
2,0.666667,0.588235,1.000000,0.610169
3,0.666667,0.588235,0.500000,0.584746
4,0.000000,0.058824,0.333333,0.042373
...,...,...,...,...
635,0.666667,0.647059,0.833333,0.661017
636,0.666667,0.705882,0.166667,0.686441
637,0.666667,0.705882,0.666667,0.711864
638,1.000000,0.882353,0.333333,0.872881


### 4.3 source & destination

In [23]:
X_train.source

0         Delhi
1         Delhi
2       Kolkata
3         Delhi
4        Mumbai
         ...   
635       Delhi
636       Delhi
637     Kolkata
638    Banglore
639       Delhi
Name: source, Length: 640, dtype: object

In [24]:
X_train.destination

0         Cochin
1         Cochin
2       Banglore
3         Cochin
4      Hyderabad
         ...    
635       Cochin
636       Cochin
637     Banglore
638        Delhi
639       Cochin
Name: destination, Length: 640, dtype: object

In [37]:
location_subset = X_train.loc[:,['source','destination']]
location_subset

,source,destination
0,Delhi,Cochin
1,Delhi,Cochin
2,Kolkata,Banglore
3,Delhi,Cochin
4,Mumbai,Hyderabad
...,...,...
635,Delhi,Cochin
636,Delhi,Cochin
637,Kolkata,Banglore
638,Banglore,Delhi


In [34]:
location_pipe1 = Pipeline(steps=[
    ('grouper', RareLabelEncoder(tol=0.1, replace_with='other', n_categories=2)),
    ('mean_encoding', MeanEncoder()),
    ("scaler", PowerTransformer())
])

location_pipe1.fit_transform(location_subset, y_train)

,source,destination
0,1.033319,1.031179
1,1.033319,1.031179
2,-0.013882,-0.064799
3,1.033319,1.031179
4,-1.749644,-0.819113
...,...,...
635,1.033319,1.031179
636,1.033319,1.031179
637,-0.013882,-0.064799
638,-1.050080,-1.743092


>- Mean Encoding : Mean encoding means lets say we have cateorgies like cat1 ,2 and 3 adn target column price.
>- So, Mean encoding takes first category and then look over all possible categories of cat1 takes corresponding price values and cal its average.
>- That average will be the mean encoding for cat1 as we calculate for each category.

>- **Importance of using these techqniue**:  Beacuse as categorical variable builds strog relation with target column that helps to improce the model performance.
>- It helps to reduce the high cardinality.
>- It can handle the ordinal features also by preserving there inherent order

>- Power Transformer : Were used to convert the data as symmetric as possible.
>- Inside power Transformer we can make standardize = True that helps to normalize the values.

In [38]:
np.union1d(
    X_train.source.unique(),
    X_train.destination.unique()
)

array(['Banglore', 'Chennai', 'Cochin', 'Delhi', 'Hyderabad', 'Kolkata',
       'Mumbai', 'New Delhi'], dtype=object)

In [54]:
def is_north(x):
    columns = x.columns.to_list()
    north_cities = ['Delhi', 'New Delhi']
    return (
        x.assign(**{
            f"{col}_is_north": x.loc[:,col].isin(north_cities).astype(int)
            for col in columns
        })
        .drop(columns=columns)
    )

FunctionTransformer(func=is_north).fit_transform(location_subset)

,source_is_north,destination_is_north
0,1,0
1,1,0
2,0,0
3,1,0
4,0,0
...,...,...
635,1,0
636,1,0
637,0,0
638,0,1


>- Functions were not able to implement in pipelines so we have to convert them as scikit learn compatible form.
>- After that we can pass function name we can easily use in pipelines.

In [56]:
location_transform = FeatureUnion(transformer_list=[
    ('part1', location_pipe1),
    ('part2', FunctionTransformer(func=is_north))
])

location_transform.fit_transform(location_subset, y_train)

,source,destination,source_is_north,destination_is_north
0,1.033319,1.031179,1,0
1,1.033319,1.031179,1,0
2,-0.013882,-0.064799,0,0
3,1.033319,1.031179,1,0
4,-1.749644,-0.819113,0,0
...,...,...,...,...
635,1.033319,1.031179,1,0
636,1.033319,1.031179,1,0
637,-0.013882,-0.064799,0,0
638,-1.050080,-1.743092,0,1


>- **FeatureUnion**: Were used to conactenate multiple transformed features together.

### 4.4 dep_time & arrival_time

In [58]:
X_train.dep_time

0      14:20:00
1      11:30:00
2      17:00:00
3      14:25:00
4      12:45:00
         ...   
635    10:20:00
636    08:00:00
637    09:35:00
638    07:10:00
639    18:15:00
Name: dep_time, Length: 640, dtype: object

In [59]:
X_train.arrival_time

0      22:30
1      19:15
2      22:05
3      19:15
4      19:25
       ...  
635    19:00
636    04:25
637    09:45
638    10:05
639    12:35
Name: arrival_time, Length: 640, dtype: object

In [61]:
time_subset = X_train[['dep_time','arrival_time']]
time_subset

,dep_time,arrival_time
0,14:20:00,22:30
1,11:30:00,19:15
2,17:00:00,22:05
3,14:25:00,19:15
4,12:45:00,19:25
...,...,...
635,10:20:00,19:00
636,08:00:00,04:25
637,09:35:00,09:45
638,07:10:00,10:05


In [77]:
time_pipe1 = Pipeline(steps=[
    ('dt', DatetimeFeatures(features_to_extract=['hour','minute'])),
    ('Mm', MinMaxScaler())
])

time_pipe1.fit_transform(time_subset)

,dep_time_hour,dep_time_minute,arrival_time_hour,arrival_time_minute
0,0.608696,0.363636,0.956522,0.545455
1,0.478261,0.545455,0.826087,0.272727
2,0.739130,0.000000,0.956522,0.090909
3,0.608696,0.454545,0.826087,0.272727
4,0.521739,0.818182,0.826087,0.454545
...,...,...,...,...
635,0.434783,0.363636,0.826087,0.000000
636,0.347826,0.000000,0.173913,0.454545
637,0.391304,0.636364,0.391304,0.818182
638,0.304348,0.181818,0.434783,0.090909


In [79]:
def part_of_day(X, morning=4, noon=12, eve=16, night=20):
    columns = X.columns.to_list()
    X_temp = X.assign(**{
        col : pd.to_datetime(X.loc[:,col]).dt.hour
        for col in columns
    })

    return (
        X_temp
        .assign(**{
            f"{col}_part_of_day": np.select(
                [X_temp.loc[:,col].between(morning, noon, inclusive='left'),
                X_temp.loc[:,col].between(noon, eve, inclusive='left'),
                X_temp.loc[:,col].between(eve, night, inclusive='left')],
                ['morning', 'afternoon', 'evening'],
                default = 'night'
            )
            for col in columns
        })
        .drop(columns=columns)
    )

FunctionTransformer(func=part_of_day).fit_transform(time_subset)

,dep_time_part_of_day,arrival_time_part_of_day
0,afternoon,night
1,morning,evening
2,evening,night
3,afternoon,evening
4,afternoon,evening
...,...,...
635,morning,evening
636,morning,morning
637,morning,morning
638,morning,morning


In [73]:
time_pipe2 = Pipeline(steps=[
    ('part', FunctionTransformer(func=part_of_day)),
    ('encoder', CountFrequencyEncoder()),
    ('scaler', MinMaxScaler())
])

time_pipe2.fit_transform(time_subset)

,dep_time_part_of_day,arrival_time_part_of_day
0,0.000000,0.85
1,1.000000,0.39
2,0.204167,0.85
3,0.000000,0.39
4,0.000000,0.39
...,...,...
635,1.000000,0.39
636,1.000000,1.00
637,1.000000,1.00
638,1.000000,1.00


>- **Count Encoding** : So in this concept each category will be replace by there occurances.
    
>- **Why we use** : 1) As we know that machine understand the numebrs thats y.
                    2) It makes simplicity to uuderstand.
                    3) As we know that using OHE may leads towards so many numbers of featurs if we have high cardinality. So this cocept works well also wont leads towards multi-collinearity.

>- **Impact** : 1) It can reatins some useful info like frequency.
                2) So it also intrdouce the high biaseness , which is not desirable.
                3) Reduces Multicollineairty.

In [74]:
time_transformer = FeatureUnion(transformer_list=[
    ('part1', time_pipe1),
    ('part2', time_pipe2)
])

time_transformer.fit_transform(time_subset)

,dep_time_hour,dep_time_minute,arrival_time_hour,arrival_time_minute,dep_time_part_of_day,arrival_time_part_of_day
0,0.608696,0.363636,0.956522,0.545455,0.000000,0.85
1,0.478261,0.545455,0.826087,0.272727,1.000000,0.39
2,0.739130,0.000000,0.956522,0.090909,0.204167,0.85
3,0.608696,0.454545,0.826087,0.272727,0.000000,0.39
4,0.521739,0.818182,0.826087,0.454545,0.000000,0.39
...,...,...,...,...,...,...
635,0.434783,0.363636,0.826087,0.000000,1.000000,0.39
636,0.347826,0.000000,0.173913,0.454545,1.000000,1.00
637,0.391304,0.636364,0.391304,0.818182,1.000000,1.00
638,0.304348,0.181818,0.434783,0.090909,1.000000,1.00


### 4.5 duration

In [80]:
X_train.duration

0       490
1       465
2      1745
3      1730
4      1840
       ... 
635     520
636    1225
637    1450
638     175
639    1100
Name: duration, Length: 640, dtype: int64

In [112]:
(
    X_train
    .duration
    .quantile([0.25,0.5,0.75])
    .values
    .reshape(-1,1)
    # .shape
)

array([[170. ],
       [497.5],
       [965. ]])

In [117]:
class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
	def __init__(self, variables=None, percentiles=[0.25, 0.5, 0.75], gamma=0.1):
		self.variables = variables
		self.percentiles = percentiles
		self.gamma = gamma


	def fit(self, X, y=None):
		if not self.variables:
			self.variables = X.select_dtypes(include="number").columns.to_list()

		self.reference_values_ = {
			col: (
				X
				.loc[:, col]
				.quantile(self.percentiles)
				.values
				.reshape(-1, 1)
			)
			for col in self.variables
		}

		return self


	def transform(self, X):
		objects = []
		for col in self.variables:
			columns = [f"{col}_rbf_{int(percentile * 100)}" for percentile in self.percentiles]
			obj = pd.DataFrame(
				data=rbf_kernel(X.loc[:, [col]], Y=self.reference_values_[col], gamma=self.gamma),
				columns=columns
			)
			objects.append(obj)
		return pd.concat(objects, axis=1)

>- For each specified column, creates new columns indicating how similar each value in the original column is to the calculated reference values (percentiles) using the RBF kernel.
>- The new columns are named based on the original column name and the percentile (e.g., col_rbf_25 for the 25th percentile).
>- Combines these new columns into a single DataFrame and returns it.

>- In summary, this class transforms numerical columns into new features that represent the similarity of each value to certain percentiles of that column using the RBF kernel method.

In [118]:
RBFPercentileSimilarity().fit_transform(X_train[['duration']])

,duration_rbf_25,duration_rbf_50,duration_rbf_75
0,0.000000,3.606563e-03,0.0
1,0.000000,1.341669e-46,0.0
2,0.000000,0.000000e+00,0.0
3,0.000000,0.000000e+00,0.0
4,0.000000,0.000000e+00,0.0
...,...,...,...
635,0.000000,1.032385e-22,0.0
636,0.000000,0.000000e+00,0.0
637,0.000000,0.000000e+00,0.0
638,0.082085,0.000000e+00,0.0


>- **RBF Kernel**: Were used to calculate the relation between two values. The input variable and target columns --> qunatiles of durations.
>- Simply we calculate the qunatiles and how close the values with this qunatiles values.
>- We are going to use Skleanrs RBF Kernel from skleanr.metrics. 
>- We have to specify the X and Y -- X : n x p --> n = number of observations and p = number of input variables. Y : m x p --> refernece values m = number of refernce values : means how many values we are going to compare with input values to calcualte RBF.

In [121]:
duration_pipe1 = Pipeline(steps=[
    ('rbf', RBFPercentileSimilarity()),
    ('scaler', PowerTransformer())
])

# duration_pipe1.fit_transform(X_train[['duration']])

duration_union = FeatureUnion(transformer_list=[
    ('part1', duration_pipe1),
    ('part2', StandardScaler())
])

duration_transformer = Pipeline(steps=[
    ('outlier',Winsorizer(capping_method='iqr', fold=1.5)),
    ('Imputer', SimpleImputer(strategy='median')),
    ('union', duration_union)
])

duration_transformer.fit_transform(X_train[['duration']])

,duration_rbf_25,duration_rbf_50,duration_rbf_75,duration
0,-0.411171,4.536830,-0.105521,-0.289409
1,-0.411171,-0.121773,-0.105521,-0.338552
2,-0.411171,-0.121773,-0.105521,2.177531
3,-0.411171,-0.121773,-0.105521,2.148046
4,-0.411171,-0.121773,-0.105521,2.364272
...,...,...,...,...
635,-0.411171,-0.121773,-0.105521,-0.230439
636,-0.411171,-0.121773,-0.105521,1.155373
637,-0.411171,-0.121773,-0.105521,1.597653
638,1.978845,-0.121773,-0.105521,-0.908602


>- **Winsorizer** : It is an outlier handling Technqiue according to the distribution of the data.
>- It is an class avialble in feature_engine.outliers import Winsorizer.

## 5. Column Transformer

In [122]:
column_transformer = ColumnTransformer(transformers=[
    ('air', air_transformer, ['airline']),
    ('doj', doj_transformer,['date_of_journey']),
    ('location', location_transform,['source','destination']),
    ('time', time_transformer, ['dep_time','arrival_time']),
    ('dur', duration_transformer, ['duration'])
    
], remainder='passthrough')

column_transformer.fit_transform(X_train, y_train)

,air__airline_Air India,air__airline_Indigo,air__airline_Jet Airways,air__airline_Multiple Carriers,air__airline_other,doj__date_of_journey_month,doj__date_of_journey_week,doj__date_of_journey_day_of_week,doj__date_of_journey_day_of_year,location__source,location__destination,location__source_is_north,location__destination_is_north,time__dep_time_hour,time__dep_time_minute,time__arrival_time_hour,time__arrival_time_minute,time__dep_time_part_of_day,time__arrival_time_part_of_day,dur__duration_rbf_25,dur__duration_rbf_50,dur__duration_rbf_75,dur__duration,remainder__total_stops,remainder__additional_info
0,0.0,1.0,0.0,0.0,0.0,1.000000,0.823529,0.500000,0.822034,1.033319,1.031179,1,0,0.608696,0.363636,0.956522,0.545455,0.000000,0.85,-0.411171,4.536830,-0.105521,-0.289409,1.0,No Info
1,0.0,0.0,0.0,1.0,0.0,1.000000,0.882353,0.833333,0.898305,1.033319,1.031179,1,0,0.478261,0.545455,0.826087,0.272727,1.000000,0.39,-0.411171,-0.121773,-0.105521,-0.338552,1.0,No Info
2,0.0,0.0,1.0,0.0,0.0,0.666667,0.588235,1.000000,0.610169,-0.013882,-0.064799,0,0,0.739130,0.000000,0.956522,0.090909,0.204167,0.85,-0.411171,-0.121773,-0.105521,2.177531,1.0,No Info
3,1.0,0.0,0.0,0.0,0.0,0.666667,0.588235,0.500000,0.584746,1.033319,1.031179,1,0,0.608696,0.454545,0.826087,0.272727,0.000000,0.39,-0.411171,-0.121773,-0.105521,2.148046,2.0,No Info
4,1.0,0.0,0.0,0.0,0.0,0.000000,0.058824,0.333333,0.042373,-1.749644,-0.819113,0,0,0.521739,0.818182,0.826087,0.454545,0.000000,0.39,-0.411171,-0.121773,-0.105521,2.364272,2.0,No Info
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,0.0,0.0,0.0,1.0,0.0,0.666667,0.647059,0.833333,0.661017,1.033319,1.031179,1,0,0.434783,0.363636,0.826087,0.000000,1.000000,0.39,-0.411171,-0.121773,-0.105521,-0.230439,1.0,No Info
636,0.0,0.0,1.0,0.0,0.0,0.666667,0.705882,0.166667,0.686441,1.033319,1.031179,1,0,0.347826,0.000000,0.173913,0.454545,1.000000,1.00,-0.411171,-0.121773,-0.105521,1.155373,1.0,In-flight meal not included
637,0.0,0.0,1.0,0.0,0.0,0.666667,0.705882,0.666667,0.711864,-0.013882,-0.064799,0,0,0.391304,0.636364,0.391304,0.818182,1.000000,1.00,-0.411171,-0.121773,-0.105521,1.597653,1.0,No Info
638,0.0,1.0,0.0,0.0,0.0,1.000000,0.882353,0.333333,0.872881,-1.050080,-1.743092,0,1,0.304348,0.181818,0.434783,0.090909,1.000000,1.00,1.978845,-0.121773,-0.105521,-0.908602,0.0,No Info


In [15]:
X_train.columns.to_list()

['airline',
 'date_of_journey',
 'source',
 'destination',
 'dep_time',
 'arrival_time',
 'duration',
 'total_stops',
 'additional_info']